In [1]:
## Boiler plate code common to many notebooks.  See the TestFilesCommonCode.ipynb for details
from __future__ import print_function
%run TestFilesCommonCode.ipynb

SimpleITK Version: 0.9.1
Compiled: Sep 28 2015 10:07:41



In [6]:
import os
import glob
import sys

#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/
#####################################################################################
#     Prepend the shell environment search paths
PROGRAM_PATHS = '/scratch/NAMICExternalProjects/release-20160523/bin'
#PROGRAM_PATHS = '/scratch/BS/release-BSR/bin'
PROGRAM_PATHS = PROGRAM_PATHS.split(':')
PROGRAM_PATHS.extend(os.environ['PATH'].split(':'))
os.environ['PATH'] = ':'.join(PROGRAM_PATHS)

CUSTOM_ENVIRONMENT=dict()

# Platform specific information
#     Prepend the python search paths
#PYTHON_AUX_PATHS = '/scratch/BS/BRAINSTools/AutoWorkup'
PYTHON_AUX_PATHS = '/scratch/SuperResolution/BRAINSSuperResolution/HCPWorkflows/'
PYTHON_AUX_PATHS = PYTHON_AUX_PATHS.split(':')
PYTHON_AUX_PATHS.extend(sys.path)
sys.path = PYTHON_AUX_PATHS

import SimpleITK as sitk
import nipype
from nipype.interfaces.base import CommandLine, CommandLineInputSpec, TraitedSpec, File, Directory
from nipype.interfaces.base import traits, isdefined, BaseInterface
from nipype.interfaces.utility import Merge, Split, Function, Rename, IdentityInterface
import nipype.interfaces.io as nio   # Data i/oS
import nipype.pipeline.engine as pe  # pypeline engine
from nipype.interfaces.freesurfer import ReconAll
from nipype.interfaces.ants import DenoiseImage
from nipype.interfaces.semtools import *

PYTHON_AUX_PATHS = '/scratch/wmql/tract_querier/tract_querier/:/scratch/wmql/tract_querier/tract_querier/nipype/'
PYTHON_AUX_PATHS = PYTHON_AUX_PATHS.split(':')
PYTHON_AUX_PATHS.extend(sys.path)
sys.path = PYTHON_AUX_PATHS

In [7]:
import numpy as np

In [11]:
# gs: gold standard
# in: input
gs_cst_left_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs_backup/WMQL/Baseline_query_cst.left.vtp'
gs_cst_right_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs_backup/WMQL/Baseline_query_cst.right.vtp'

sr_cst_left_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs_backup/WMQL/Baseline_query_cst.left.vtp'
sr_cst_right_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs_backup/WMQL/Baseline_query_cst.right.vtp'

In [12]:
from tractography import Tractography, tractography_to_file, tractography_from_files
from wmql import TractQuerier

In [13]:
#tr_files = [gs_cst_left_fn, in_cst_left_fn]
tractography = tractography_from_files(gs_cst_left_fn)

In [24]:
print(tractography)
tracts = tractography.tracts()
print(len(tracts))
tractography_points = np.vstack(tracts)
print(tractography_points.min(0))

<tractography.tractography.Tractography instance at 0x133b455f0>
619
[-62.87464905 -81.971138   -64.76567841]


In [15]:
lengths = [len(tracts[i]) for i in xrange(len(tracts))]
print(lengths)
print(max(lengths))

[140, 88, 103, 110, 111, 132, 75, 112, 111, 132, 128, 96, 96, 215, 106, 112, 108, 109, 95, 114, 111, 98, 130, 96, 96, 104, 98, 104, 116, 133, 127, 132, 91, 109, 116, 106, 99, 105, 109, 104, 135, 97, 96, 113, 105, 107, 111, 152, 105, 92, 123, 121, 123, 99, 112, 101, 97, 107, 129, 118, 131, 119, 111, 109, 78, 99, 77, 121, 97, 87, 102, 66, 93, 107, 99, 142, 110, 135, 80, 123, 85, 103, 100, 101, 130, 104, 123, 137, 100, 118, 110, 99, 104, 125, 111, 58, 89, 69, 119, 113, 104, 110, 89, 133, 96, 103, 118, 65, 114, 131, 121, 99, 109, 111, 104, 106, 97, 107, 116, 131, 111, 104, 112, 87, 102, 115, 107, 119, 131, 142, 113, 109, 109, 86, 110, 101, 103, 103, 111, 94, 96, 104, 107, 110, 128, 105, 128, 97, 85, 108, 98, 94, 95, 97, 114, 109, 83, 113, 87, 95, 85, 100, 93, 107, 114, 96, 117, 114, 99, 102, 111, 103, 106, 124, 106, 106, 101, 64, 111, 126, 100, 120, 126, 104, 95, 98, 128, 95, 96, 115, 110, 102, 138, 83, 93, 135, 134, 79, 103, 108, 121, 111, 103, 111, 110, 107, 105, 108, 93, 104, 104, 110, 

In [16]:
def CSTOverlap(gs_cst_left, gs_cst_right, sr_cst_left, sr_cst_right):
    # gs: gold standard, sr: super-resolution reconstructed
    import os
    #########
    def ComputeBhattacharyyaCoeficient(baseline_bundle, sr_bundle):
        from tractography import Tractography, tractography_to_file, tractography_from_files
        import numpy as np
        ##
        def ReturnDistributionInEachCoordinate(tractography_points):
            from scipy import stats
            x_arr = tractography_points[:,0]
            y_arr = tractography_points[:,1]
            z_arr = tractography_points[:,2]
            x = np.linspace(x_arr.min(), x_arr.max(), 100)
            y = np.linspace(y_arr.min(), y_arr.max(), 100)
            z = np.linspace(z_arr.min(), z_arr.max(), 100)
            kde_x = stats.gaussian_kde(x_arr)
            kde_y = stats.gaussian_kde(y_arr)
            kde_z = stats.gaussian_kde(z_arr)
            p_x = kde_x(x)
            p_y = kde_y(y)
            p_z = kde_z(z)
            return p_x, p_y, p_z
        ## read in each fiber bundle
        tractography_gs = tractography_from_files(baseline_bundle)
        tracts_gs = tractography_gs.tracts()
        tractography_points_gs = np.vstack(tracts_gs)
        #
        tractography_sr = tractography_from_files(sr_bundle)
        tracts_sr = tractography_sr.tracts()
        tractography_points_sr = np.vstack(tracts_sr)
        ## Use ksdensity to estimate probability density function for the sample data in each coordinate
        [p_gs_x, p_gs_y, p_gs_z] = ReturnDistributionInEachCoordinate(tractography_points_gs)
        [p_sr_x, p_sr_y, p_sr_z] = ReturnDistributionInEachCoordinate(tractography_points_sr)
        BC = (1.0/3.0)*( np.sum(np.sqrt(p_gs_x * p_sr_x)) + np.sum(np.sqrt(p_gs_y * p_sr_y)) + np.sum(np.sqrt(p_gs_z * p_sr_z)) )
        return BC
        ##
    def writeLabelStatistics(filename,statsList):
        import csv
        label = os.path.splitext(os.path.basename(filename))[0]
        with open(filename, 'wb') as lf:
            headerdata = [['#Label', 'BC_left.cst', 'BC_right.cst', 'BC_cst']]
            wr = csv.writer(lf, delimiter=',')
            wr.writerows(headerdata)
            wr.writerows([[label] + statsList])
    #########
    # compute Bhattacharyya Coeficient for cst.left/right/total
    bc_l = ComputeBhattacharyyaCoeficient(gs_cst_left,sr_cst_left)
    if bc_l > 1: bc_l = 1
    bc_r = ComputeBhattacharyyaCoeficient(gs_cst_right,sr_cst_right)
    if bc_r > 1: bc_r = 1
    bc_total = (bc_l + bc_r)/2.0
    statsList = [format(bc_l,'.4f'), format(bc_r,'.4f'), format(bc_total,'.4f')]
    # create output file name
    srfn = os.path.basename(sr_cst_left)
    srfnbase = os.path.splitext(srfn)[0]
    srLabel = srfnbase.split('_',1)[0]
    fn = srLabel + '_BhattacharyyaCoeficient.csv'
    output_csv_file = os.path.join(os.getcwd(), fn)
    # write the stats list
    writeLabelStatistics(output_csv_file,statsList)
    assert os.path.isfile(output_csv_file), "Output Bhattacharyya coeficient file is not found: %s" % output_csv_file
    return output_csv_file

In [17]:
fn = CSTOverlap(gs_cst_left_fn, gs_cst_right_fn, sr_cst_left_fn, sr_cst_right_fn)
print(fn)

/Volumes/scratch/Ipython_conda/SimpleITK-Notebook-Answers/Baseline_BhattacharyyaCoeficient.csv


In [ ]:
import pandas as pd
from ggplot import *

In [ ]:
DF_gs_x = pd.DataFrame(np.transpose([x, p_gs_x]))
DF_gs_x.rename(columns = lambda x: str(x), inplace=True)
DF_gs_x.rename(columns={"0": "X"}, inplace=True) # rename a dfcolumn   
DF_gs_x.rename(columns={"1": "Y"}, inplace=True) # rename a dfcolumn 
#DF_gs_x

ggplot(aes(x = 'X', y ='Y'),data=DF_gs_x) + \
    geom_point()

In [ ]:
DF_in_x = pd.DataFrame(np.transpose([x, p_in_x]))
DF_in_x.rename(columns = lambda x: str(x), inplace=True)
DF_in_x.rename(columns={"0": "X"}, inplace=True) # rename a dfcolumn   
DF_in_x.rename(columns={"1": "Y"}, inplace=True) # rename a dfcolumn 
#DF_in_x

ggplot(aes(x = 'X', y ='Y'),data=DF_in_x) + \
    geom_point()

# Bhattacharyya Coefficient

In [ ]:
Bc = (1.0/3.0)*( np.sum(np.sqrt(p_gs_x * p_in_x)) + np.sum(np.sqrt(p_gs_y * p_in_y)) + np.sum(np.sqrt(p_gs_z * p_in_z)) )
print(Bc)

In [ ]:
a = np.array([1,2,3])
b = np.array([4,5,6])

In [ ]:
np.vstack((a,b))